In [8]:
import time
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from utils import *
from model import *
from process import *
import pickle
import itertools
import pickle
import os

In [2]:
data='cora'
adj, features, labels,idx_train,idx_val,idx_test = load_citation(data)
#splitstr = 'splits/'+data+'_split_0.6_0.2_'+str(0)+'.npz'
#adj, features, labels, idx_train, idx_val, idx_test, num_features, num_labels = full_load_data(data,splitstr)
print(adj.shape)
print(features.shape)
print(labels.shape)
print(idx_train.shape)
print(idx_val.shape)
print(idx_test.shape)

torch.Size([2708, 2708])
torch.Size([2708, 1433])
torch.Size([2708])
torch.Size([140])
torch.Size([500])
torch.Size([1000])


C:\Users\ayday\Desktop\Master_Thesis\experiment\utils.py:87: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))


In [3]:
cudaid = "cuda"
device = torch.device(cudaid)
features = features.to(device)
adj = adj.to(device).coalesce()

In [4]:
n=features.shape[0]
feature_size=features.shape[1]
num_classes = len(torch.unique(labels))




In [14]:
class my_GraphConvolution(nn.Module):
    

    def __init__(self, in_features, out_features,nfeat, bias=True):
        super(my_GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features+n+nfeat, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj,x):
        
        
        #support = torch.cat((input-torch.mm(adj.to_dense(), input), adj.to_dense()),1)
        
        support = torch.cat((torch.mm(adj.to_dense(), input),adj.to_dense(),x),1)
        output = torch.mm(support, self.weight)

        #output= torch.mm(torch.mm(adj.to_dense(), input) + input, self.weight)

       
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'
        
class my_GCN(nn.Module):
    def __init__(self, n,nfeat, nhid_list, nclass, dropout):
        super(my_GCN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if nhid_list:
            # Input layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nhid_list[0],nfeat))
            
            # Hidden layers
            for i in range(1, len(nhid_list)):
                self.layers.append(my_GraphConvolution(nhid_list[i-1], nhid_list[i],nfeat))
            
            # Output layer
            self.layers.append(my_GraphConvolution(nhid_list[-1], nclass,nfeat))
        else:
            # Single output layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nclass,nfeat))
        
        self.dropout = dropout

    def forward(self, x, adj):
        h = x.detach().requires_grad_()
        
        for i, layer in enumerate(self.layers[:-1]):
            h = F.relu(layer(h, adj,x))
            h = F.dropout(h, self.dropout, training=self.training)
        
        h = self.layers[-1](h, adj,x)
        
        return F.log_softmax(h, dim=1)



model=my_GCN(n,feature_size,[512,256],num_classes,dropout=0.5)
model=model.to(device)

optimizer = optim.Adam(model.parameters(),lr=0.1,weight_decay=5e-4)

epochs=100
patience=10
test=False

In [37]:
model

my_GCN(
  (layers): ModuleList(
    (0): my_GraphConvolution (1433 -> 512)
    (1): my_GraphConvolution (512 -> 7)
  )
)

In [87]:
def train():
    model.train()
    optimizer.zero_grad()
    output = model(features,adj)
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()
    return loss_train.item(),acc_train.item()


def validate():
    model.eval()
    with torch.no_grad():
        output = model(features,adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(),acc_val.item()

def test():
    #model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(),acc_test.item()
    
t_total = time.time()
bad_counter = 0
best = 999999999
best_epoch = 0
acc = 0
for epoch in range(epochs):
    loss_tra,acc_tra = train()
    loss_val,acc_val = validate()
    if(epoch+1)%1 == 0: 
        print('Epoch:{:04d}'.format(epoch+1),
            'train',
            'loss:{:.3f}'.format(loss_tra),
            'acc:{:.2f}'.format(acc_tra*100),
            '| val',
            'loss:{:.3f}'.format(loss_val),
            'acc:{:.2f}'.format(acc_val*100))
    if loss_val < best:
        best = loss_val
        best_epoch = epoch
        acc = acc_val
        #torch.save(model.state_dict(), checkpt_file)
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == patience:
        break

if test:
    acc = test()[1]

print("Train cost: {:.4f}s".format(time.time() - t_total))
print('Load {}th epoch'.format(best_epoch))
print("Val","acc.:{:.1f}".format(acc*100))

Epoch:0001 train loss:2.006 acc:15.00 | val loss:27.952 acc:5.80
Epoch:0002 train loss:30.248 acc:15.71 | val loss:3.002 acc:22.40
Epoch:0003 train loss:5.137 acc:33.57 | val loss:2.377 acc:26.80
Epoch:0004 train loss:1.925 acc:51.43 | val loss:1.543 acc:27.60
Epoch:0005 train loss:0.630 acc:82.86 | val loss:1.334 acc:58.20
Epoch:0006 train loss:0.273 acc:98.57 | val loss:1.279 acc:60.40
Epoch:0007 train loss:0.134 acc:100.00 | val loss:1.179 acc:65.80
Epoch:0008 train loss:0.091 acc:100.00 | val loss:1.111 acc:65.20
Epoch:0009 train loss:0.062 acc:100.00 | val loss:1.210 acc:62.40
Epoch:0010 train loss:0.034 acc:100.00 | val loss:1.562 acc:50.80
Epoch:0011 train loss:0.045 acc:100.00 | val loss:1.175 acc:65.60
Epoch:0012 train loss:0.024 acc:100.00 | val loss:1.030 acc:64.40
Epoch:0013 train loss:0.032 acc:100.00 | val loss:1.388 acc:51.60
Epoch:0014 train loss:0.038 acc:100.00 | val loss:1.446 acc:57.00
Epoch:0015 train loss:0.056 acc:99.29 | val loss:1.018 acc:64.40
Epoch:0016 train

In [92]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import itertools

# Assuming my_GraphConvolution is already defined somewhere

class my_GCN(nn.Module):
    def __init__(self, nfeat, nhid_list, nclass, dropout):
        super(my_GCN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if nhid_list:
            # Input layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nhid_list[0],nfeat))
            
            # Hidden layers
            for i in range(1, len(nhid_list)):
                self.layers.append(my_GraphConvolution(nhid_list[i-1], nhid_list[i],nfeat))
            
            # Output layer
            self.layers.append(my_GraphConvolution(nhid_list[-1], nclass,nfeat))
        else:
            # Single output layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nclass,nfeat))
        
        self.dropout = dropout

    def forward(self, x, adj):
        h = x.detach().requires_grad_()
        
        for i, layer in enumerate(self.layers[:-1]):
            h = F.relu(layer(h, adj,x))
            h = F.dropout(h, self.dropout, training=self.training)
        
        h = self.layers[-1](h, adj,x)
        
        return F.log_softmax(h, dim=1)

# Assuming accuracy function is already defined

def train(model, optimizer, features, adj, labels, idx_train):
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()
    return loss_train.item(), acc_train.item()

def validate(model, features, adj, labels, idx_val):
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(), acc_val.item()

def test(model, features, adj, labels, idx_test, checkpt_file):
    model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(), acc_test.item()

# Hyperparameters
nfeat = features.shape[1]
nclass = num_classes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hyperparams = {
    'lr': [0.01, 0.005],
    'weight_decay': [5e-4, 5e-3],
    'dropout': [0.5, 0.3],
    'nhid_list': [[], [256, 512], [16, 32, 16], [64, 128, 64], [128, 256, 128], [512, 1024, 512]]

}

# To store results
results = []

for lr, weight_decay, dropout, nhid_list in itertools.product(
    hyperparams['lr'], hyperparams['weight_decay'], hyperparams['dropout'], hyperparams['nhid_list']
):
    model = my_GCN(nfeat, nhid_list, nclass, dropout)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    epochs = 100
    patience = 10
    best = 999999999
    best_epoch = 0
    acc = 0
    t_total = time.time()
    bad_counter = 0

    for epoch in range(epochs):
        loss_tra, acc_tra = train(model, optimizer, features, adj, labels, idx_train)
        loss_val, acc_val = validate(model, features, adj, labels, idx_val)
        if (epoch+1) % 1 == 0: 
            '''
            print('Epoch:{:04d}'.format(epoch+1),
                'train',
                'loss:{:.3f}'.format(loss_tra),
                'acc:{:.2f}'.format(acc_tra*100),
                '| val',
                'loss:{:.3f}'.format(loss_val),
                'acc:{:.2f}'.format(acc_val*100))
            '''
        if loss_val < best:
            best = loss_val
            best_epoch = epoch
            acc = acc_val
            #torch.save(model.state_dict(), checkpt_file)
            bad_counter = 0
        else:
            bad_counter += 1

        if bad_counter == patience:
            break
    
    #print("Train cost: {:.4f}s".format(time.time() - t_total))
    #print('Load {}th epoch'.format(best_epoch))
    #print("Val", "acc.:{:.1f}".format(acc*100))

    results.append({
        'lr': lr,
        'weight_decay': weight_decay,
        'dropout': dropout,
        'nhid_list': nhid_list,
        'accuracy': acc
    })

# Sort results by accuracy in descending order
results = sorted(results, key=lambda x: x['accuracy'], reverse=True)

print("Hyperparameters and accuracies sorted by accuracy:")
for result in results:
    print(result)


Hyperparameters and accuracies sorted by accuracy:
{'lr': 0.005, 'weight_decay': 0.0005, 'dropout': 0.5, 'nhid_list': [128, 256, 128], 'accuracy': 0.786}
{'lr': 0.005, 'weight_decay': 0.0005, 'dropout': 0.3, 'nhid_list': [256, 512], 'accuracy': 0.78}
{'lr': 0.01, 'weight_decay': 0.0005, 'dropout': 0.3, 'nhid_list': [256, 512], 'accuracy': 0.778}
{'lr': 0.01, 'weight_decay': 0.005, 'dropout': 0.5, 'nhid_list': [512, 1024, 512], 'accuracy': 0.778}
{'lr': 0.01, 'weight_decay': 0.0005, 'dropout': 0.3, 'nhid_list': [64, 128, 64], 'accuracy': 0.776}
{'lr': 0.005, 'weight_decay': 0.005, 'dropout': 0.3, 'nhid_list': [128, 256, 128], 'accuracy': 0.776}
{'lr': 0.005, 'weight_decay': 0.005, 'dropout': 0.3, 'nhid_list': [64, 128, 64], 'accuracy': 0.774}
{'lr': 0.01, 'weight_decay': 0.0005, 'dropout': 0.5, 'nhid_list': [64, 128, 64], 'accuracy': 0.772}
{'lr': 0.005, 'weight_decay': 0.005, 'dropout': 0.5, 'nhid_list': [512, 1024, 512], 'accuracy': 0.772}
{'lr': 0.005, 'weight_decay': 0.005, 'dropout

In [ ]:
{'lr': 0.005, 'weight_decay': 0.0005, 'dropout': 0.3, 'nhid_list': [16, 32], 'accuracy': 0.778}
{'lr': 0.005, 'weight_decay': 0.0005, 'dropout': 0.5, 'nhid_list': [128, 256, 128], 'accuracy': 0.786}

In [17]:
class my_GraphConvolution(nn.Module):
    

    def __init__(self, in_features, out_features,nfeat, bias=True):
        super(my_GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features+n+nfeat, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj,x):
        
        
        #support = torch.cat((input-torch.mm(adj.to_dense(), input), adj.to_dense()),1)
        
        support = torch.cat((torch.mm(adj.to_dense(), input),adj.to_dense(),x),1)
        output = torch.mm(support, self.weight)

        #output= torch.mm(torch.mm(adj.to_dense(), input) + input, self.weight)

       
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class my_GCN(nn.Module):
    def __init__(self, nfeat, nhid_list, nclass, dropout):
        super(my_GCN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if nhid_list:
            # Input layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nhid_list[0],nfeat))
            
            # Hidden layers
            for i in range(1, len(nhid_list)):
                self.layers.append(my_GraphConvolution(nhid_list[i-1], nhid_list[i],nfeat))
            
            # Output layer
            self.layers.append(my_GraphConvolution(nhid_list[-1], nclass,nfeat))
        else:
            # Single output layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nclass,nfeat))
        
        self.dropout = dropout

    def forward(self, x, adj):
        h = x.detach().requires_grad_()
        
        for i, layer in enumerate(self.layers[:-1]):
            h = F.relu(layer(h, adj,x))
            h = F.dropout(h, self.dropout, training=self.training)
        
        h = self.layers[-1](h, adj,x)
        
        return F.log_softmax(h, dim=1)

def train(model, optimizer, features, adj, labels, idx_train):
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()
    return loss_train.item(), acc_train.item()

def validate(model, features, adj, labels, idx_val):
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(), acc_val.item()

def test(model, features, adj, labels, idx_test, checkpt_file):
    model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(), acc_test.item()
        
# Hyperparameters
nfeat = features.shape[1]
nclass = num_classes
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
hyperparams = {
    'lr': [0.01],
    'weight_decay': [5e-4],
    'dropout': [0.5,0.3],
    'nhid_list': [[]],
}

def load_previous_results(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    return []

def save_results(file_path, results):
    with open(file_path, 'wb') as f:
        pickle.dump(results, f)

# File paths for saving results and combinations
results_file_path = 'results.pkl'
combinations_file_path = 'combinations.pkl'

# To store results
results = []

# To store already run combinations
run_combinations = set()

# Load previously run combinations if available
# For simplicity, assuming results is a list of dicts loaded from a file
results = load_previous_results(results_file_path)
for result in results:
     run_combinations.add((result['lr'], result['weight_decay'], result['dropout'], tuple(result['nhid_list'])))

for lr, weight_decay, dropout, nhid_list in itertools.product(
    hyperparams['lr'], hyperparams['weight_decay'], hyperparams['dropout'], hyperparams['nhid_list']
):
    combination = (lr, weight_decay, dropout, tuple(nhid_list))
    
    if combination in run_combinations:
        print(f"Skipping already run combination: {combination}")
        continue
    
    run_combinations.add(combination)
    
    model = my_GCN(nfeat, nhid_list, nclass, dropout)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    epochs = 100
    patience = 10
    best = 999999999
    best_epoch = 0
    acc = 0
    t_total = time.time()
    bad_counter = 0

    for epoch in range(epochs):
        loss_tra, acc_tra = train(model, optimizer, features, adj, labels, idx_train)
        loss_val, acc_val = validate(model, features, adj, labels, idx_val)
        if (epoch+1) % 1 == 0: 
            
            print('Epoch:{:04d}'.format(epoch+1),
                'train',
                'loss:{:.3f}'.format(loss_tra),
                'acc:{:.2f}'.format(acc_tra*100),
                '| val',
                'loss:{:.3f}'.format(loss_val),
                'acc:{:.2f}'.format(acc_val*100))
            
        if loss_val < best:
            best = loss_val
            best_epoch = epoch
            acc = acc_val
            #torch.save(model.state_dict(), checkpt_file)
            bad_counter = 0
        else:
            bad_counter += 1

        if bad_counter == patience:
            break
    
    print("Train cost: {:.4f}s".format(time.time() - t_total))
    print('Load {}th epoch'.format(best_epoch))
    print("Val", "acc.:{:.1f}".format(acc*100))

    results.append({
        'lr': lr,
        'weight_decay': weight_decay,
        'dropout': dropout,
        'nhid_list': nhid_list,
        'accuracy': acc
    })

# Sort results by accuracy in descending order
results = sorted(results, key=lambda x: x['accuracy'], reverse=True)

print("Hyperparameters and accuracies sorted by accuracy:")
for result in results:
    print(result)

# Save results and run_combinations for future use
save_results(results_file_path, results)
save_results(combinations_file_path, list(run_combinations))

Skipping already run combination: (0.01, 0.0005, 0.5, ())
Epoch:0001 train loss:1.966 acc:15.71 | val loss:1.910 acc:21.80
Epoch:0002 train loss:1.932 acc:17.86 | val loss:1.904 acc:21.80
Epoch:0003 train loss:1.899 acc:22.86 | val loss:1.899 acc:22.60
Epoch:0004 train loss:1.866 acc:25.71 | val loss:1.894 acc:23.40
Epoch:0005 train loss:1.833 acc:30.71 | val loss:1.889 acc:23.20
Epoch:0006 train loss:1.801 acc:37.14 | val loss:1.883 acc:24.60
Epoch:0007 train loss:1.770 acc:45.71 | val loss:1.878 acc:26.60
Epoch:0008 train loss:1.739 acc:54.29 | val loss:1.872 acc:27.80
Epoch:0009 train loss:1.708 acc:62.14 | val loss:1.867 acc:29.40
Epoch:0010 train loss:1.678 acc:69.29 | val loss:1.861 acc:31.20
Epoch:0011 train loss:1.648 acc:74.29 | val loss:1.856 acc:32.60
Epoch:0012 train loss:1.619 acc:80.71 | val loss:1.850 acc:34.40
Epoch:0013 train loss:1.591 acc:86.43 | val loss:1.845 acc:35.40
Epoch:0014 train loss:1.563 acc:88.57 | val loss:1.840 acc:37.80
Epoch:0015 train loss:1.536 acc:

In [9]:

print(adj.to_dense().shape)
        print(support.shape)
        print(output.shape)


torch.Size([2708, 2708]) adj
torch.Size([2708, 16])  support= input x weight
torch.Size([2708, 2724])

IndentationError: unexpected indent (1557039151.py, line 2)